In [1]:
from typing import Dict

import toolpy as tp
from toolpy.integrations import groq

In [2]:
groq_interface = groq.GroqInterface(model=groq.GroqModel.LLAMA3_70B, n_retry=5)

registry = tp.llm.LLMRegistry()
registry.registry(model_name="llama3-70b", interface=groq_interface, default=True)

In [3]:
class OutlineGenerator(tp.BasicTool):
    _description = "Creates informative text ouline."

    _system_message = '''You are a informative article outline generator that outputs in JSON. 
The JSON object must use the schema: {'article':[{'section_name':'str', 'description':'str'}, {'section_name':'str', 'description':'str'}, ...]},

Please use a valid JSON format.'''

    _base_prompt = '''Generate a draft of article sections for the instructions:

Theme: {theme}
Audience: {audience}
'''

    _return_description = {"article": "list of article sections, with names and descriptions"}

    _input_description = {"theme":"theme of the article", "audience":"intended audience for the article"}

    def __init__(self, model_name: str | None = None) -> None:
        super().__init__(self._description, self._input_description, 
                         self._base_prompt, self._return_description, 
                         self._system_message, model_name, True)

In [11]:
with open("Seminar transcription - Only presentation.txt", "r") as file:
    seminar_transcription = file.readlines()

seminar_transcription_filtered = []

for line in seminar_transcription:
    if line == "" or line == "\n":
        continue

    seminar_transcription_filtered.append(line)

seminar_transcription = " ".join(seminar_transcription_filtered)

In [18]:
context = f'''
Seminar Transcription:
{seminar_transcription}

Personal keypoints:
- Generative AI is better defined as a negative classification: AI that is not for classificattion, regression or control.
- There is ideological discousers around generative AI, like about emergentism, escaling laws, Turing test, antropomorphism and Shannon divide.
- We should be carefull about metaphors around generative AI, like they can "understand", "think", "reasonate", "halicinate". They can complicate or mislead the models analysis. 
- Shannon stated that semantic is irrelevant for the engineering problem of language models. Only the previous sentence words are important to describe the next word probability.
- Shannon divide: is semantic irrelevant? Is dificult to reintroduce semantic, and we would need a "theory of semantics" for that.
- Emergentism: the phenomenon is not described as the sum of parts. It can be a mirage and there is a not undestanded incremental performance improvement.
- Escaling laws: greater the model, greater the performance. But it may not be getting better at what matter for the users (crossentropy is not language habilities).
- Turing test: if mislead a human, it must be inteligent. But what about non-human intelligence capabilities?
- Antropomorphism: makes us let the guard down about what this models and the companies behind they does. Is a design choice.
- Tips for gen AI usage includes: few shot, instruction tuning (not the first choice, is expensive), don't train from the 0 (is really expensive), be aware of the bias and of the hype.

The informative article must use the above provided information. 
In special, the personal keypoints guide the most important parts of the seminar.
'''

query = {"theme":"prompt engineering", "audience":"general public"}

In [6]:
outline_generator = OutlineGenerator()

In [19]:
result, _ = outline_generator(query, context)

KeyboardInterrupt: 

In [8]:
result

{'article': [{'section_name': 'What is Prompt Engineering?',
   'description': 'Introduction to the concept of prompt engineering, its importance, and relevance in AI development'},
  {'section_name': 'The Role of Language in AI',
   'description': 'Explaining how language and communication play a crucial role in AI systems and how prompt engineering comes into play'},
  {'section_name': 'Types of Prompts',
   'description': 'Overview of different types of prompts, including input prompts, output prompts, and primal prompts, and their applications'},
  {'section_name': 'The Art of Crafting Effective Prompts',
   'description': 'Tips and best practices for crafting high-quality prompts that elicit desired responses from AI models'},
  {'section_name': 'Real-World Applications of Prompt Engineering',
   'description': 'Examples and case studies of prompt engineering in action, including its use in chatbots, virtual assistants, and content generation'},
  {'section_name': 'Challenges and 